In [ ]:
import os
from matplotlib import pyplot
import numpy as np
import h5py

In [ ]:
def add_to_plots(ax, quantity, h5file, name):
    ds = h5file[quantity]
    noise = ds.attrs['noiselevels']
    avg = np.average(ds[...], axis=1)
    maybe_std = np.std(ds[...], axis=1)
    ycenter, spread, yerr  = avg.T
    if np.any(np.isnan(maybe_std)):
        ax[0].plot(noise, spread, label=name)
        ax[1].plot(noise, ycenter, label=name)
        ax[2].plot(noise, yerr, label=name)
    else:
        maybe_std = maybe_std * 5 / 4 / np.sqrt(5)
        ax[0].errorbar(noise, spread, yerr=maybe_std[:,0], label=name, capsize=10.)
        ax[1].errorbar(noise, ycenter, yerr=maybe_std[:,1], label=name, capsize=10.)
        ax[2].errorbar(noise, yerr, yerr=maybe_std[:,2], label=name, capsize=10.)
    ax[0].legend()
    if quantity == 'pose':
        ax[0].set(xlim=(0.,65), ylim=(0.,3.), ylabel='Spread / °')
        ax[1].set(xlim=(0.,65), ylim=(5.2,5.9), ylabel='rot loc')
        ax[2].set(xlim=(0.,65), ylim=(5.,6.5), xlabel='Input noise std', ylabel='rot err')
    elif quantity == 'roi':
        ax[0].set(xlim=(0.,70), ylim=(0.,0.02), ylabel='coordinate spread [%]')
        ax[1].set(xlim=(0.,70), ylim=(0.,0.2), ylabel='coordinate loc error [%]')
        ax[2].set(xlim=(0.,70), ylim=(0.,0.2), xlabel='input noise std', ylabel='roi coordinate error [%]')

In [ ]:
files = [
    # Generated by evaluate_stability.py --noise-resist
    '../model_files/pub2_baseline/pub2_baseline_mobilenet_noise.h5',
    '../model_files/pub2_baseline/pub2_baseline_resnet18_noise.h5',
    '../model_files/pub2_synface/pub2_synface_mobilenet_noise.h5',
    '../model_files/pub2_synface/pub2_synface_resnet18_noise.h5',
]

In [ ]:
for quantity in ['pose']:
    fig, ax = pyplot.subplots(3,1,dpi=120,figsize=(5,4))
    for filename in files:
        with h5py.File(filename,'r') as f:
            metrics = add_to_plots(ax, quantity, f, os.path.basename(filename))

pyplot.show()
fig.savefig("/tmp/noise_resist_tmp.svg")